In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Let's get some info about the data we will be working with.
* train.csv  is the training data (store_nbr, family, sales, promotion and date )
 store_nbr : store number or store ID
 family identifies the group a product belongs to ex. automotivr, sport, food, beverage, etc..
 onPromotion :  number of articles on promotion of a given family
 sales : total amount of sales for product/family/store at a given date
 
* test.cvs is testing data with the same structure of train.csv
* store.csav : store_nbr	city	state	type	cluster
* We are asking you to predict total sales for every product/ store in the next month.

### Defining a path variable to simplify coding

In [ ]:
from path import Path
path = Path('/kaggle/input/store-sales-time-series-forecasting/')

### Reading data from files

In [ ]:
train_sales = pd.read_csv(path/'train.csv')
test_sales = pd.read_csv(path/'test.csv')
shops = pd.read_csv(path/'stores.csv')
holidays = pd.read_csv(path/'holidays_events.csv')
transaction = pd.read_csv(path/'transactions.csv')

### Let's do some Data Analisys

In [ ]:
train_sales.head(5)

In [ ]:
train_sales.info()

In [ ]:
train_sales

In [ ]:
print("Number of Time Series (train):")
unique_keys_train = set(zip(train_sales.store_nbr, train_sales.family))
print(len(unique_keys_train))



In [ ]:
train_sales.family.unique()

In [ ]:
train_sales.store_nbr.unique()

In [ ]:
train_sales.corr()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import seaborn as sns

correlation = train_sales.corr()
max_correlation = correlation.index[abs(correlation["sales"])>0.01]
plt.figure(figsize=(10,10))
graph=sns.heatmap(train_sales[max_correlation].corr(), annot=True, cmap="YlGnBu")

In [ ]:
shops.head(5)

In [ ]:
len(shops)

In [ ]:
holidays

In [ ]:
transaction

### Let's Plot Total Sales of all Stores

In [ ]:
# Total sales of all stores
from learntools.time_series.style import *   #  plot style setings

total_sales_all_stores = train_sales.groupby('date').sum()['sales']
ax = total_sales_all_stores.plot(**plot_params)
ax.set(title="Total sales of all stores")

In [ ]:
#x  Total sales  x  store number

total_sales_storenumber = train_sales.groupby('store_nbr').sum()['sales']
sx = total_sales_storenumber.plot(**plot_params)
sx.set(title="Total Sales x Store Number ")


In [ ]:
#x  Total sales  x  product family

total_sales_family = train_sales.groupby('family').sum()['sales']
fx = total_sales_family.plot(**plot_params)
fx.set(title="Total Sales x Product's Family ")

In [ ]:
train_sales.family.unique()

In [ ]:
train_sales.groupby('family').sum()['sales']

### Let's try to do a kind of sql join
* Using the value sales.store_nbr which is the column for store_id in the table ( file ) sales
* we are going to access the table shops which also has store_id as one of its columns and then
* from that table we will get the column city which is the city where the store is located

In [ ]:
train_sales['city']= train_sales.store_nbr.map(shops.city)
train_sales['city']


###  Let's take a look at   Total Sales by Date and by  City

In [ ]:
sales_by_city= train_sales.groupby(['date',train_sales['city']])['sales'].sum().reset_index()
sales_by_city = sales_by_city.set_index('date')
sales_by_city

###  Total Sales by City

In [ ]:
 train_sales.groupby('city').sum()['sales']

In [ ]:
#x  Total sales  x  city

total_sales_city = train_sales.groupby('city').sum()['sales']
fx = total_sales_city.plot(**plot_params)
fx.set(title="Total Sales x City ")

### OK now let break down the column date in to Year, Month, Week and Day
* This will be helpful to analyze the data and build some nice Graphs

In [ ]:
import calendar
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

train_sales['date'] = pd.to_datetime(train_sales['date'])
train_sales['year'] = train_sales['date'].dt.year
train_sales['month']= train_sales['date'].dt.month
train_sales['week']= train_sales['date'].dt.isocalendar().week
train_sales['day_of_week']= train_sales['date'].dt.day_name()

In [ ]:
total_sales_month = train_sales.groupby('month').sum()['sales']
sx = total_sales_month.plot(**plot_params)
sx.set(title="Total Sales x Month ")

In [ ]:
total_sales_dayweek = train_sales.groupby('day_of_week').sum()['sales']
fdw = total_sales_dayweek.plot(**plot_params)
fdw.set(title='Total Sales by Day of the Week')


In [ ]:
total_sales_dayweek = train_sales.groupby('day_of_week').agg({"sales": "mean"}).reset_index()
fdw = total_sales_dayweek.plot(**plot_params)
fdw.set(title='Average Sales by Day of the Week')

In [ ]:
# data
df_dw_sa = train_sales.groupby('day_of_week').agg({"sales" : "mean"}).reset_index()
df_dw_sa.sales = round(df_dw_sa.sales, 2)

# chart
fig = px.bar(df_dw_sa, y='day_of_week', x='sales', title='Avg Sales vs Day of Week',
             color_discrete_sequence=['#c6ccd8'], text='sales',
             category_orders=dict(day_of_week=["Monday","Tuesday","Wednesday","Thursday", "Friday","Saturday","Sunday"]))
fig.update_yaxes(showgrid=False, ticksuffix=' ', showline=False)
fig.update_xaxes(visible=False)
fig.update_layout(margin=dict(t=60, b=0, l=0, r=0), height=350,
                  hovermode="y unified", 
                  yaxis_title=" ", template='plotly_white',
                  title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#c6ccd8", font_size=13, font_family="Lato, sans-serif"))

In [ ]:

#offline.init_notebook_mode(connected = True)

fig=go.Figure()
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3],
    y=[1.6, 1.6, 1.6, 1.6],
    mode="text", 
    text=["<span style='font-size:33px'><b>54</b></span>", 
          "<span style='font-size:33px'><b>33</b></span>",
          "<span style='font-size:33px'><b>16</b></span>",
          "<span style='font-size:33px'><b>56</b></span>"],
    textposition="bottom center"
))
fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3],
    y=[1.1, 1.1, 1.1, 1.1],
    mode="text", 
    text=["Stores", "Products", "States", "Months"],
    textposition="bottom center"
))
fig.add_hline(y=2.2, line_width=5, line_color='gray')
fig.add_hline(y=0.3, line_width=3, line_color='gray')
fig.update_yaxes(visible=False)
fig.update_xaxes(visible=True)
fig.update_layout(showlegend=False, height=300, width=700, 
                  title='Store Sales Summary', title_x=0.5, title_y=0.9,
                  xaxis_range=[-0.5,3.6], yaxis_range=[-0.2,2.2],
                  plot_bgcolor='#fafafa', paper_bgcolor='#fafafa',
                  font=dict(size=23, color='#323232'),
                  title_font=dict(size=35, color='#222'),
                  margin=dict(t=90,l=70,b=0,r=70), 
    )

In [ ]:
train_sales[:2]

In [ ]:

sf =train_sales.groupby(['store_nbr', train_sales['family']])
print(sf)


In [ ]:
# Slice [start:stop:step], starting from index 5 take every 6th record.
train_df = train_sales[5::6]
#date_time = pd.to_datetime(tain_df.pop('Date Time'), format=''%d.%m.%y')
train_df.head()

In [ ]:
train_df.describe().transpose()

### Let's do One Hot Encoding ..

In [ ]:
train_sales_features = ['store_nbr', 'family','sales']
train_sales_one_hot = train_sales[train_sales_features]
train_sales_one_hot.head()

In [ ]:
train_sales_one_hot = pd.get_dummies(train_sales_one_hot, drop_first=True)

In [ ]:
train_sales.head()

In [ ]:
train_sales_one_hot.tail()

In [ ]:
# train_sales_one_hot['date'] = str(train_sales_one_hot['date'])

## OK, let's split the data

In [ ]:
column_indices = {name: i for i, name in enumerate(train_sales_one_hot.columns)}
nl = len(train_sales_one_hot)
train_df = train_sales_one_hot[0:int(nl*0.85)]    # int(0.85 * len(nl))
val_df =  train_sales_one_hot[0:int(nl*0.15)]      #  int(0.15 * len(nl))
num_features = train_sales_one_hot.shape[1]

print("train_df :", train_df)


###  Now, Let's Normalize the data
* It is important to scale features before training a neural network. Normalization is a common way of doing this scaling: subtract the mean and divide by the standard deviation of each feature.

* The mean and standard deviation should only be computed using the training data so that the models have no access to the values in the validation and test sets.

In [ ]:
train_df['sales'] = (train_df['sales'] - train_df['sales'].mean())/train_df['sales'].std()

In [ ]:
print("raw_data : ", train_df['sales'])

In [ ]:
train_df.tail()

## Let's creates some functions ( tools ) 
* I am going to use Tensorflow timeseries weather forcasting as a guide :  https://www.tensorflow.org/tutorials/structured_data/time_series#split_the_data

## 1. Creates window Generator:
* Start by creating the WindowGenerator class. The __init__ method includes all the necessary logic for the input and label indices.
* It also takes the training, evaluation, and test DataFrames as input. These will be converted to tf.data.Datasets of windows later. 



In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
#  1.  Windows Generator

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_sales,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])



## Let's creates some functions ( tools ) 

## 2. Spliting the Data
* Given a list of consecutive inputs, the split_window method will convert them to a window of inputs and a window of labels.


In [ ]:
# 2.  Spliting windows

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

​
## 3.  Create Dataset
* Finally, this make_dataset method will take a time series DataFrame and convert it to a tf.data.Dataset of (input_window, label_window) pairs using the tf.keras.utils.timeseries_dataset_from_array function:
* The WindowGenerator object holds training, validation, and test data.
* Add properties for accessing them as tf.data.Datasets using the make_dataset method you defined earlier. Also, add a standard example batch for easy access and plotting:



In [ ]:

# 3.  create Dataset.

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example



## 4. Compile and Fit the model.
* Parameters model, window, max-epochs, learning_rate, patience
* Define loss function, optimizer as well as the metrics
* Fit.model parameters input dataset, epochs, validation dataset and callbacks
* Return history

In [ ]:
# 4.  Compile and Fit the model.

def compile_and_fit(model, window, MAX_EPOCHS=20, learning_rate=0.2, patience=10):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
                metrics=[tf.metrics.MeanAbsoluteError()])
    
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    
    return history

## Data windowing
* We will make a set of predictions based on a window of consecutive samples from the data.

* The main features of the input windows are:

1. The **width** (number of time steps) of the input and label windows.
2. The **time offset** between them. ( shift )
3. Which **features ( label_width )** are used as inputs, labels, or both.
This tutorial builds a variety of models (including Linear, DNN, CNN and RNN models), and uses them for both:

Single-output, and multi-output predictions.
Single-time-step and multi-time-step predictions.

For example, to make a single prediction 24 hours into the future, given 24 hours of history, you might define a window like this:

One prediction 24 hours into the future.

[0][1][2] ..... [23][24][25] .........[46][47]
input_width = 24
offset = 24
label_width = 1
total_width = 48e

In [ ]:
window1 = WindowGenerator(input_width=24,     # 24 hours history
                          label_width = 1,   # one single prediction
                          shift=24,          # 24 hours in to the future
                         label_columns=['sales'])
window1

In [ ]:
# Stack three slices, the length of the total window.
train_df= tf.convert_to_tensor(train_df, np.float32)
example_window = tf.stack([np.array(train_df[:window1.total_window_size]),
                           np.array(train_df[100:100+window1.total_window_size]),
                           np.array(train_df[200:200+window1.total_window_size])])

example_inputs, example_labels = window1.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
example_window.example = example_inputs, example_labels

### Let's  create a wider WindowGenerator that generates windows 30 days of consecutive inputs and labels at a time.
* The new wide_window variable doesn't change the way the model operates.
* The model still makes predictions one day into the future based on 30 day ( one week ) inputs time step. 
* Here, the time axis acts like the batch axis: each prediction is made independently with no interaction between time steps:

In [ ]:
window30 = WindowGenerator(
    input_width=30,
    label_width=1,
    shift=1,
    label_columns=['sales'])

window30

In [ ]:
# Stack three slices, the length of the total window.
train_df= tf.convert_to_tensor(train_df, np.float32)
example_window30 = tf.stack([np.array(train_df[:window30.total_window_size]),
                           np.array(train_df[100:100+window30.total_window_size]),
                           np.array(train_df[200:200+window30.total_window_size])])

example_inputs, example_labels = window30.split_window(example_window30)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window30.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

In [ ]:
example_window30.example = example_inputs, example_labels

### Typically, data in TensorFlow is packed into arrays
* where the outermost index is across examples (the "batch" dimension).
* The middle indices are the "time" or "space" (width, height) dimension(s). The innermost indices are the features.

* The code above took a batch of three 48-time step windows with 34 features at each time step. It splits them into a batch of 26-time step 19-feature inputs, and a 1-time step 1-feature label. The label only has one feature because the WindowGenerator was initialized with label_columns=['T (degC)']. Initially, this tutorial will build models that predict single output labels.

In [ ]:
def plot(self, model=None, plot_col='sales', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
window1.plot()

In [ ]:
window1.train.element_spec

In [ ]:
window1.val.element_spec

In [ ]:
window30.plot()

In [ ]:
for example_inputs, example_labels in window1.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

### Create Baseline model

In [ ]:
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [ ]:
window30.val

In [ ]:
baseline = Baseline(label_index = column_indices['sales'])
baseline.compile(loss=tf.losses.MeanSquaredError(),
                metrics = [tf.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(window30.val)
#performance['Baseline'] = baseline.evaluate(window30.test, verbose=0)

In [ ]:
print('Input shape:', window30.example[0].shape)
print('Output shape:', baseline(window30.example[0]).shape)

### Ok now I am going to try using favorite book Deep Learning with Python Time Series example.

### We will use the time series dataset from array() to instatiate three datasets ( Data Objects )
* One for training, one for validation and one for testing.
* Note that each **dataset is a tuple ( samples, targets )** samples(batch_size,sequence_length )
* **sample_rate = 6** Observation will be sampled at one datapoint per hour. We will only keep one datapoint out of 6
* **sequence_length = 120** Observations will go back 5 days ( 120 hours )
* **delay** = sampling_rate * ( sequence_length + 24 - 1 ) the target for a sequence will be the temperature 24 hours after the end of the sequence.
* **train_dataset start_index = 0**, end_index=num_train_samples we will take all num_train_samples
* **val_dataset** start_index= num_train_samples end_index= num_train_samples + num_val_samples we will take all num_val_samples after num_train_samples
* **test_dataset** : start_index= num_train_samples + num_val_samples we will take all the rest of datapoint after train_dataset and val_dataset
* **batch_size = 256** target temperatures
* The samples are randomly shuffled, so two consecutive sequences in a batch (like samples[0] and samples[1] ) aren't necessarily temporally close.

### OK Let's Split the data
* We calculate the number of records for training, validation and testing
* In this particular case training = 80%, validation = 15% and the rest for training = 5%

In [ ]:
num_train_samples = int(0.80 * len(train_sales_one_hot))   # number of record for training 80%
num_val_samples = int(0.15 * len(train_sales_one_hot))     # number of records for validation 15%
num_test_samples = len(train_sales_one_hot) - num_train_samples - num_val_samples

print("Num_train_samples :", num_train_samples)
print("num_val_samples : ", num_val_samples)
print("num_test_samples : ", num_test_samples)

### I am going to use the following features:
*  Date,  store_nbr,   family,  sales
* To keep the original date unchanged I will use another variable to do the one hot encoding  ( train_sales_one_hot  )
*  I also convert date timestamp to string 

In [ ]:
train_sales_features = ['date','store_nbr', 'family','sales']
train_sales_one_hot = train_sales[train_sales_features]
train_sales_one_hot['date'] = str(train_sales_one_hot['date'])
train_sales_one_hot.head()

### Let's do One Hot Encoding ..

In [ ]:
# Slice [start:stop:step], starting from index 30 take every 31th record.
# train_df = train_sales_one_hot[30::31]
train_df = train_sales_one_hot
#date_time = pd.to_datetime(tain_df.pop('Date Time'), format=''%d.%m.%y')
train_df.head()

### Let's normalize sales 

In [ ]:
train_df['sales'] = (train_df['sales'] - train_df['sales'].mean())/train_df['sales'].std()

###  Let's take a look at the standarized data

In [ ]:
train_df.describe().transpose()

In [ ]:
len(train_df)

### OK Let's now do one hot encoding 

In [ ]:
train_df_one_hot = pd.get_dummies(train_df, drop_first=True)

In [ ]:
train_df_one_hot.head()

### We will use the time series dataset from array() to instatiate three datasets ( Data Objects )
* One for training, one for validation and one for testing.
* Note that each **dataset is a tuple ( samples, targets )** samples(batch_size,sequence_length )
* **sample_rate = 1** Observation will be sampled at one datapoint per day. We will only keep one datapoint out of 1
* **sequence_length = 300** Observations will go back 30 days 
* **delay** = sampling_rate * ( sequence_length + 24 - 1 ) the target for a sequence will be the sales 30 days after the end of the sequence.
* **train_dataset start_index = 0**, end_index=num_train_samples we will take all num_train_samples
* **val_dataset** start_index= num_train_samples end_index= num_train_samples + num_val_samples we will take all num_val_samples after num_train_samples
* **test_dataset** : start_index= num_train_samples + num_val_samples we will take all the rest of datapoint after train_dataset and val_dataset
* **batch_size = 256** target temperatures
* The samples are randomly shuffled, so two consecutive sequences in a batch (like samples[0] and samples[1] ) aren't necessarily temporally close.

In [ ]:
from posixpath import devnull
from tensorflow import keras 
import numpy as np

sampling_rate = 1
sequence_length = 30
delay = sampling_rate * (sequence_length + 1 - 1)
batch_size =256
train_target = train_df_one_hot['sales']
# Trainig Dataset.. first 50% of raw_data
train_dataset = keras.utils.timeseries_dataset_from_array(  # train_dataset ( samples,targets) samples(256,120,14) targets(256,)
    train_df_one_hot[:-delay],
    targets = train_target[delay:],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    shuffle = True,
    batch_size = batch_size,
    start_index = 0,
    end_index = num_train_samples
)
#  Validation dataset .. next 25% of raw_data
val_dataset = keras.utils.timeseries_dataset_from_array(
    train_df_one_hot[:-delay],
    targets=train_target[delay:],
    sampling_rate = sampling_rate,
    sequence_length=  sequence_length,
    shuffle  = True,
    batch_size = batch_size,
    start_index = num_train_samples,
    end_index = num_train_samples + num_val_samples
)
# Testing dataset .. last 25% of raw data
test_dataset = keras.utils.timeseries_dataset_from_array(
    train_df_one_hot[:-delay],
    targets = train_target[delay:],
    sampling_rate = sampling_rate,
    sequence_length = sequence_length,
    shuffle = True,
    batch_size = batch_size,
    start_index = num_train_samples + num_val_samples
)



In [ ]:
for samples, targets in train_dataset:
  print("samples shape : ", samples.shape)
  print("targets shape : ", targets.shape)
  print(" Len samples : ",len(samples) )
  break

In [ ]:
len(train_dataset)

In [ ]:
from tensorflow.keras import layers

# 1.  Build the model

inputs = tf.keras.Input(shape=(sequence_length,  train_df_one_hot.shape[-1] ))
x = tf.keras.layers.Flatten() ( inputs )
x = tf.keras.layers.Dense(16, activation ='relu') (x)
outputs = tf.keras.layers.Dense(1) (x)
model = tf.keras.Model(inputs, outputs)

callbacks =[
    tf.keras.callbacks.ModelCheckpoint("jena_dense_keras",
                                      save_best_only=True)
    
]

# 2.  Compile the model

model.compile(
    tf.keras.optimizers.RMSprop(),
    tf.keras.losses.mse,
    metrics = ['mae'])

# 3. Fit / Train the model

history = model.fit(
    train_dataset,
    epochs = 4,
    validation_data = val_dataset,
    callbacks = callbacks)

model = tf.keras.models.load_model("jena_dense_keras")
print(f"test MAE : {model.evaluate(test_dataset)[1]:2f}")

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) +1)
plt.figure()
plt.plot(epochs, loss,"bo", label="Training MAE")
plt.plot(epochs, val_loss,"b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()